# Boat Object Detection with YOLO

In this exercise, you will fine-tune a YOLO model for boat detection in satellite imagery.

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
import pandas as pd

## Data Collection

Download the training and test datasets.

In [ ]:
# URLs for the dataset files
data_train_features_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/X_train.pkl'
data_train_labels_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/y_train.pkl'
data_test_features_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/X_test.pkl'

def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name}')

# Download the files
download_file(data_train_features_url, 'X_train.pkl')
download_file(data_train_labels_url, 'y_train.pkl')
download_file(data_test_features_url, 'X_test.pkl')

In [ ]:
# Load the training data
with open('X_train.pkl','rb') as f: X_train = pickle.load(f)
with open('y_train.pkl','rb') as f: y_train = pickle.load(f)
with open('X_test.pkl','rb')  as f: X_test  = pickle.load(f)


print(f"Loaded {len(X_train)} training images")
print(f"Sample image shape: {getattr(X_train[0], 'shape', 'unknown')}")
print(f"First label type/len: {type(y_train[0])}, {len(y_train[0]) if hasattr(y_train[0],'__len__') else 'n/a'}")
print("Example first label item (up to 3 boxes):", (y_train[0][:3] if hasattr(y_train[0],'__len__') else y_train[0]))

## Data Visualization

Display sample images with bounding boxes.

In [ ]:
def plot_image_with_boxes(image, boxes, ax=None):
    """
    Plot an image with bounding boxes
    boxes: numpy array with shape (n, 5) - [class_id, x_center, y_center, width, height]
    All box coordinates are normalized (0-1)
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    ax.imshow(image)
    height, width = image.shape[:2]

    # Draw each bounding box
    for box in boxes:
        class_id, x_center, y_center, box_width, box_height = box

        # Convert from normalized YOLO format to pixel coordinates
        x_center_px = x_center * width
        y_center_px = y_center * height
        box_width_px = box_width * width
        box_height_px = box_height * height

        # Calculate top-left corner
        x1 = x_center_px - box_width_px / 2
        y1 = y_center_px - box_height_px / 2

        # Draw rectangle
        rect = patches.Rectangle(
            (x1, y1), box_width_px, box_height_px,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)

        # Add label
        ax.text(x1, y1-5, 'boat', color='red', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    ax.axis('off')
    return ax

In [ ]:
# Display 5 sample images with bounding boxes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(6):
    plot_image_with_boxes(X_train[i], y_train[i], ax=axes[i])
    axes[i].set_title(f'Image {i}: {len(y_train[i])} boat(s)')

plt.tight_layout()
plt.show()

## Model Setup

Load a pre-trained YOLO model and test it on the training images.

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model (nano version)
model = YOLO('yolov8n.pt')

print("Model loaded successfully!")

In [ ]:
from sklearn.model_selection import train_test_split
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
print(len(X_train), len(X_val))
print(len(y_train), len(y_val))

In [ ]:
# Test the pre-trained model on a sample image
# Save a sample image temporarily
from PIL import Image
sample_img = Image.fromarray(X_train2[0])
sample_img.save('temp_sample.png')

# Run prediction
results = model.predict(source='temp_sample.png', conf=0.25)

print(f"Pre-trained model detected {len(results[0].boxes)} objects")
print("Note: The pre-trained model may not detect boats well - you need to fine-tune it!")

## Fine-tune your YOLO model

Your task is to fine-tune the YOLO model on the boat detection dataset to achieve a mAP50 score of at least 0.73 on the test set.

In [ ]:
# Your code here
# Fine-tune the YOLO model


In [ ]:
def labels_to_yolo_lines(boxes, img_width=640, img_height=480):
    lines = []
    for box in boxes:
        cls, x_min, y_min, x_max, y_max = box
        # conversion en coordonnées normalisées YOLO
        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        lines.append(f"{int(cls)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return lines


In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image

# --- util ---
def to_uint8_rgb(arr):
    arr = np.asarray(arr)
    if arr.dtype != np.uint8:
        # suppose arr in [0,1] ou [0,255]
        if arr.max() <= 1.0:
            arr = (arr * 255.0).clip(0,255).astype(np.uint8)
        else:
            arr = arr.clip(0,255).astype(np.uint8)
    if arr.ndim == 2:
        arr = np.stack([arr]*3, axis=-1)
    if arr.shape[-1] == 4:
        arr = arr[..., :3]
    return arr

def _is_normalized(vals):
    return np.all((vals >= 0.0) & (vals <= 1.0))

def labels_to_yolo_lines(boxes, img_shape=None):
    """
    Accepte plusieurs formats d'entrée pour `boxes`:
      - YOLO normalisé: [cls, cx, cy, w, h]  -> passe-plat
      - YOLO en pixels: [cls, cx, cy, w, h]  -> normalise avec img_shape
      - XYXY pixels   : [cls, xmin, ymin, xmax, ymax] -> convertit en YOLO normalisé
    """
    boxes = np.asarray(boxes, dtype=float)
    if boxes.ndim == 1:
        boxes = boxes[None, :]

    H = W = None
    if img_shape is not None:
        H, W = img_shape[:2]

    lines = []
    for b in boxes:
        if len(b) < 5:
            # ignore silencieusement
            continue

        cls = int(round(b[0]))
        v = b[1:5]

        # cas 1: déjà YOLO normalisé (cx,cy,w,h in [0,1])
        if _is_normalized(v):
            cx, cy, bw, bh = v

        else:
            # si pas normalisé:
            if W is None or H is None:
                raise ValueError("labels_to_yolo_lines: img_shape requis pour normaliser des valeurs en pixels.")


            xmin, ymin, xmax, ymax = v
            if xmax > xmin and ymax > ymin:
                bw = xmax - xmin
                bh = ymax - ymin
                cx = xmin + bw/2
                cy = ymin + bh/2
            else:
                # suppose YOLO en pixels (cx,cy,w,h)
                cx, cy, bw, bh = v

            # normaliser
            cx, cy, bw, bh = cx / W, cy / H, bw / W, bh / H

        # clamp léger
        cx = float(np.clip(cx, 0, 1))
        cy = float(np.clip(cy, 0, 1))
        bw = float(np.clip(bw, 0, 1))
        bh = float(np.clip(bh, 0, 1))

        lines.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
    return lines


from sklearn.model_selection import train_test_split
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

dataset_root = Path('submarine_dataset')
for split in ['train','val']:
    (dataset_root/split/'images').mkdir(parents=True, exist_ok=True)
    (dataset_root/split/'labels').mkdir(parents=True, exist_ok=True)

# TRAIN
for idx in range(len(X_train2)):
    img_np = to_uint8_rgb(X_train2[idx])
    img = Image.fromarray(img_np).convert("RGB")
    stem = f"img_{idx:06d}"
    img.save(dataset_root/'train'/'images'/f"{stem}.jpg", quality=95)

    lines = labels_to_yolo_lines(y_train2[idx], img_shape=img_np.shape)
    (dataset_root/'train'/'labels'/f"{stem}.txt").write_text("\n".join(lines))

# VAL
for idx in range(len(X_val)):
    img_np = to_uint8_rgb(X_val[idx])
    img = Image.fromarray(img_np).convert("RGB")
    stem = f"img_{idx:06d}"
    img.save(dataset_root/'val'/'images'/f"{stem}.jpg", quality=95)

    lines = labels_to_yolo_lines(y_val[idx], img_shape=img_np.shape)
    (dataset_root/'val'/'labels'/f"{stem}.txt").write_text("\n".join(lines))

print("✅ Dataset re-prepared.")
print(f"  Train: {len(X_train2)} images")
print(f"  Val:   {len(X_val)} images")


In [ ]:
# Create dataset config file for YOLO (submarines)
from pathlib import Path

dataset_root = Path('submarine_dataset')

config = f"""
path: {dataset_root.absolute()}
train: train/images
val: val/images

nc: 1
names: ['submarine']
"""

with open('submarine.yaml', 'w') as f:
    f.write(config)

print('✅ Config file created: submarine.yaml')

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import random


dataset_root = Path('submarine_dataset')

# Choisir une image aléatoire dans le train
train_images = list((dataset_root / 'train' / 'images').glob('*.jpg'))
sample_img = random.choice(train_images)

# Trouver le fichier label correspondant
sample_label = dataset_root / 'train' / 'labels' / sample_img.name.replace('.jpg', '.txt')

# Lire l'image
img = cv2.imread(str(sample_img))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

# Lire et dessiner les bounding boxes YOLO
if sample_label.exists():
    with open(sample_label) as f:
        for line in f:
            cls, x_c, y_c, width, height = map(float, line.strip().split())

            # Conversion coordonnées normalisées → pixels
            x_c, y_c, width, height = x_c * w, y_c * h, width * w, height * h
            x1 = int(x_c - width / 2)
            y1 = int(y_c - height / 2)
            x2 = int(x_c + width / 2)
            y2 = int(y_c + height / 2)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"boat", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.title(f"Sample Training Image: {sample_img.name}")
plt.axis('off')
plt.show()

In [ ]:
from ultralytics import YOLO


model_ft = YOLO('yolov8s.pt')

# Entraîner sur ton dataset
results = model_ft.train(
    data='submarine.yaml',     #  fichier de config
    epochs=50,
    imgsz=640,
    batch=8,
    name='submarine_detector', # nom du run dans runs/detect/
    patience=5,
    save=True,
    verbose=True,
    device=0,
    pretrained = True
)

print('🚀 Training complete!')

In [ ]:

model_ft = YOLO('runs/detect/submarine_detector2/weights/best.pt')
metrics = model_ft.val(data='submarine.yaml')
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

## Generate Predictions for Test Set

Generate predictions on the test set and save them in the required CSV format.

In [ ]:
# Load test data
with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

print(f"Loaded {len(X_test)} test images")

In [ ]:
# Generate predictions on test set
# Your trained model should be used here
# Format: CSV with columns: image_id, box_idx, class_id, confidence, x_center, y_center, width, height

predictions = []

for img_idx, img in enumerate(X_test):
    # Save image temporarily
    temp_img = Image.fromarray(img)
    temp_img.save('temp_test.png')

    # Run prediction with your fine-tuned model
    # Replace 'model' with your fine-tuned model
    results = model_ft.predict(source='temp_test.png', conf=0.25, verbose=False)

    # Extract predictions
    for box_idx, box in enumerate(results[0].boxes):
        # Get box data in YOLO format (normalized coordinates)
        x_center, y_center, width, height = box.xywhn[0].cpu().numpy()
        confidence = box.conf[0].cpu().numpy()
        class_id = int(box.cls[0].cpu().numpy())

        predictions.append({
            'image_id': img_idx,
            'box_idx': box_idx,
            'class_id': 0,  # Always 0 for boat
            'confidence': float(confidence),
            'x_center': float(x_center),
            'y_center': float(y_center),
            'width': float(width),
            'height': float(height)
        })

    if (img_idx + 1) % 10 == 0:
        print(f"Processed {img_idx + 1}/{len(X_test)} images")

print(f"\nTotal predictions: {len(predictions)}")

In [ ]:
# Save predictions to CSV
df_predictions = pd.DataFrame(predictions)
df_predictions.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")
print("\nFirst few predictions:")
print(df_predictions.head(10))

In [ ]:
from google.colab import files

# Télécharger le fichier
files.download('predictions.csv')